In [9]:
import random
import csv
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# Python script for confusion matrix creation.

def ANN():

    def load_csv(filename):
        with open(filename, 'r') as file:
            lines = csv.reader(file)
            dataset=list(lines)
    #    print(dataset)
        return dataset

    # Convert string column to float
    def str_column_to_float(dataset, column):
        for row in dataset:
            row[column] = float(row[column])

    # Convert string column to integer
    def str_column_to_int(dataset, column):
        class_values = [row[column] for row in dataset]
        unique = set(class_values)
        lookup = dict()
        for i, value in enumerate(unique):
            lookup[value] = i
        for row in dataset:
            row[column] = lookup[row[column]]
       # print(lookup)
       #print(dataset)
        return lookup

    # Find the min and max values for each column
    def dataset_minmax(dataset):
        minmax = list()
        stats = [[min(column), max(column)] for column in zip(*dataset)]
        return stats

    # Rescale dataset columns to the range 0-1
    def normalize_dataset(dataset, minmax):
        for row in dataset:
            for i in range(len(row)-1):
                row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

    # Split a dataset into k folds
    def cross_validation_split(dataset, n_folds):
        dataset_split = list()
        dataset_copy = list(dataset)
        fold_size = int(len(dataset) / n_folds)
        for i in range(n_folds):
            fold = list()
            while len(fold) < fold_size:
                index = randrange(len(dataset_copy))
                fold.append(dataset_copy.pop(index))
            dataset_split.append(fold)
        return dataset_split

    # Calculate accuracy percentage
    def accuracy_metric(actual, predicted):
        correct = 0
        for i in range(len(actual)):
            if actual[i] == predicted[i]:
                correct += 1
        return correct / float(len(actual)) * 100.0

    # Initialize a network
    def initialize_network(n_inputs, n_hidden1, n_hidden2, n_outputs):
            #print("No. of  Nuerons in Input layer: ", n_inputs)
            #print("No. of  Nuerons in Hidden Layer 1: ", n_hidden1)
            #print("No. of  Nuerons in Hidden Layer 2: ", n_hidden2)
            #print("No. of  Nuerons in Output layer: ", n_outputs)
            network = list()
            hidden_layer1 = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
            network.append(hidden_layer1)
            hidden_layer2 = [{'weights':[random() for i in range(n_hidden1 + 1)]} for i in range(n_hidden2)]
            network.append(hidden_layer2)
            output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
            network.append(output_layer)
            #print(network)
            return network

    # Calculate neuron activation for an input
    def activate(weights, inputs):
        activation = weights[-1]
        for i in range(len(weights)-1):
            activation += weights[i] * inputs[i]       #Complete the missing statement
        return activation


    #Complete the missing function to Transfer neuron activation

    def transfer(activation):
        return 1.0 / (1.0 + exp(-activation))

    # Forward propagate input to a network output
    def forward_propagate(network, row):
        inputs = row
        for layer in network:
            new_inputs = []
            for neuron in layer:
                activation = activate(neuron['weights'], inputs)
                neuron['output'] = transfer(activation)
                new_inputs.append(neuron['output'])
            inputs = new_inputs
        return inputs

    # Calculate the derivative of an neuron output
    def transfer_derivative(output):
        return output * (1.0 - output)

    # Backpropagate error and store in neurons
    def backward_propagate_error(network, expected):
        for i in reversed(range(len(network))):
            layer = network[i]
            errors = list()
            #For Hidden layer
            if i != len(network)-1:
                for j in range(len(layer)):
                    error = 0.0
                    for neuron in network[i + 1]:
                        error += (neuron['weights'][j] * neuron['delta'])
                    errors.append(error)
            #For Output layer
            else:
                for j in range(len(layer)):
                    neuron = layer[j]
                    errors.append(expected[j] - neuron['output'])
            for j in range(len(layer)):
                neuron = layer[j]
                neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

    # Update network weights with error
    def update_weights(network, row, l_rate): # introduced bug
        for i in range(len(network)):
            inputs = row[:-1]
            if i != 0:
                inputs = [neuron['output'] for neuron in network[i - 1]]
            for neuron in network[i]:
                for j in range(len(inputs)):
                    neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
                neuron['weights'][-1] += l_rate * neuron['delta']

    # Train a network for a fixed number of epochs
    def train_network(network, train, l_rate, n_epoch, n_outputs):
        for epoch in range(n_epoch):
            for row in train:
                outputs = forward_propagate(network, row)
                expected = [0 for i in range(n_outputs)]
                expected[row[-1]] = 1
                backward_propagate_error(network, expected)
                update_weights(network, row, l_rate)

     # Make a prediction with a network
    def predict(network, row):
        outputs = forward_propagate(network, row)
        return outputs.index(max(outputs))

    # Backpropagation Algorithm With Stochastic Gradient Descent
    def applying(train, test, l_rate, n_epoch, n_hidden):
        n_inputs = len(train[0]) - 1
        n_outputs = len(set([row[-1] for row in train]))
        network = initialize_network(n_inputs, n_hidden, n_hidden, n_outputs)
        train_network(network, train, l_rate, n_epoch, n_outputs)
        predictions = list()
        for row in test:
            prediction = predict(network, row)
            predictions.append(prediction)
        return(predictions)

    #Using all the functions
    seed(1)
    #Loading and preprocessing data
    filename = 'leaf.csv'
    dataset = load_csv(filename)
    for i in range(len(dataset[0])-1):
        str_column_to_float(dataset, i)
    #Converting class column to integers
    look = str_column_to_int(dataset, len(dataset[0])-1)
    #print(look)
    #Normalize input variables
    minmax = dataset_minmax(dataset)
    normalize_dataset(dataset, minmax)
    n_folds = 5
    l_rate = 0.3
    n_epoch = 100
    n_hidden = 5
    folds = cross_validation_split(dataset, n_folds)
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
            actual = [row[-1] for row in fold]
    #print(actual)
    print()
    #for i in [0, 10, 100, 500, 1000, 10000]:       # For Training Dataset
    #for i in [0.01, 0.2, 0.0001, 0.5, 1]:
    result=applying(train_set,test_set, l_rate , n_epoch ,n_hidden)
    #print(result)
    acc=accuracy_metric(actual,result)
    check = ''
    check = "Accuracy : " + str("{0:.2f}".format(acc * 5)) + "\n"
    matrix = []
    for i in range(40):
        l = list()
        for j in range(40):
            l.append(0)
        matrix.append(l)
    for i in range(len(actual)):
        matrix[int(actual[i] - 1)][int(result[i] - 1)] += 1
    check2 = ''
    for i in matrix:
        check2 += str(i) + "\n"
    from tkinter import messagebox
 
    messagebox.showinfo("Confusion Matrix", check2)

    ann = tk.Tk()
    ann.geometry("300x400")
    ann.resizable(0,0)
    v = tk.IntVar()

    languages = [
        ("Test",1),
    ]

    def ShowChoice():
        if (v.get() == 0):
            image_accept()
         

    tk.Label(ann, 
             text=check,
             justify = tk.CENTER, fg="#383a39", bg="#a1dbcd", font=("Helvetica", 16),
             padx = 20).pack()

    for val, language in enumerate(languages):
        tk.Radiobutton(ann, 
                      text=language,
                      indicatoron = 0,
                      width = 100,
                      padx = 20, 
                      variable=v, 
                      command=ShowChoice,
                      value=val).pack(anchor=tk.W)
    ann.mainloop()

In [2]:
def kNN():
    import csv
    import random
    import math
    import operator
    from pprint import pprint

    random.seed(5)
    def dataload(file,split,train=[],test=[]):
        with open(file,'r') as csvfile:
            lines=csv.reader(csvfile)
            data=list(lines)

            for x in range(len(data)-1):
                for y in range(38):	
                    data[x][y]=float(data[x][y])
                if(random.random()<split):
                    train.append(data[x])
                else:
                    test.append(data[x])

    # DONE
    def Edistance(x1,x2,length):
        s = 0
        #print(x1)
        for i in range(length):
            s = s + (x1[i]-x2[i] )**2
        return s**0.5

    def Mdistance(x1,x2,length):
        s = 0
        for i in range(length):
            s = s + (abs(x1[i]-x2[i]))
        return s

    def Cdistance(x1,x2,length):
        s = 0
        x1Square = [_**2 for _ in x1[:-1]]
        x2Square = [_**2 for _ in x2[:-1]]
        x1Sum = sum(x1Square)
        x2Sum = sum(x2Square)
        prod = 0
        for i in range(length):
            prod = prod + (x1[i]*x2[i])
        sim = prod/((x1Sum**0.5) * (x2Sum**0.5))
        return math.acos(sim)/math.pi

    def distance(x1,x2,length,dist):
        if(dist is "E"):
            return Edistance(x1,x2,length)
        if(dist is "M"):
            return Mdistance(x1,x2,length)
        if(dist is "C"):
            return Cdistance(x1,x2,length)

    def Neighbors(train, test, k,distType):
        distances = []
        length = len(test)-1

        for x in range(len(train)):
            dist = distance(test, train[x], length,distType)
            distances.append((train[x], dist))
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
        for x in range(k):
            neighbors.append(distances[x][0])
        return neighbors

    def Response(neighbors):
        Votes = {}
        #pprint(neighbors)
        for x in range(len(neighbors)):
            response = neighbors[x][-1]
            if response in Votes:
                Votes[response] += 1
            else:
                Votes[response] = 1
        sortedVotes = sorted(Votes.items(), key=operator.itemgetter(1), reverse=True)
        #print(sortedVotes)
        return sortedVotes[0][0]

    #Calculating Accuracy
    def Accuracy(testSet, predictions):
        correct = 0
        for x in range(len(testSet)):
            if testSet[x][-1] == predictions[x]:
                correct += 1
        return (correct/float(len(testSet))) * 100.0

    def calculation(test,trainingSet,k,distType):
        predictions = list()
        for x in range(len(test)):
            neighbors = Neighbors(trainingSet, test[x], k,distType)
            result = Response(neighbors)
            predictions.append(result)
            #print('Predicted=',result,', Actual=',test[x][-1])
        accuracy =Accuracy(test, predictions)
        return " K = " + str(k) + " : " + str("{0:.2f}".format(accuracy * 3)) + "\n"

    def confusion(test,trainingSet,k,distType,matrix):
        predictions = list()
        for x in range(len(test)):
            neighbors = Neighbors(trainingSet, test[x], k,distType)
            result = Response(neighbors)
            predictions.append(result)
        for i in range(len(test)):
            matrix[int(test[i][-1]-1)][int(predictions[i]-1)] += 1
    
    trainingSet=[]
    test=[]
    split = 0.7
    dataload('leaf.csv', split, trainingSet, test)
    predictions=[]
    distType = "M"
    check = "Train set: " + str(len(trainingSet)) + "\n" + "Test set: " + str(len(test)) + "\n"
    for k in range(1,10):
        check += calculation(test,trainingSet,k,distType)
    matrix = []
    for i in range(40):
        l = list()
        for j in range(40):
            l.append(0)
        matrix.append(l)
    confusion(test,trainingSet,1,distType,matrix)
    check2 = ''
    for i in matrix:
        check2 += str(i) + "\n"
        
    """master2 = tk.Tk()

    w = tk.Message(master2, text=check2)
    w.pack()

    master2.mainloop()"""
    
    from tkinter import messagebox
 
    messagebox.showinfo("Confusion Matrix", check2)

    knn = tk.Tk()
    knn.geometry("300x400")
    knn.resizable(0,0)
    v = tk.IntVar()

    languages = [
        ("Test",1),
    ]

    def ShowChoice():
        if (v.get() == 0):
            image_accept()
         

    tk.Label(knn, 
             text=check,
             justify = tk.CENTER,fg="#383a39", bg="#a1dbcd", font=("Helvetica", 16),
             padx = 20).pack()

    for val, language in enumerate(languages):
        tk.Radiobutton(knn, 
                      text=language,
                      indicatoron = 0,
                      width = 100,
                      padx = 20, 
                      variable=v, 
                      command=ShowChoice,
                      value=val).pack(anchor=tk.W)
    knn.mainloop()

In [3]:
def image_accept():
    import tkinter as tk
    knn2 = tk.Tk()
    #knn2.geometry("300x600")
    #knn2.resizable(0,0)
    global var
    var = tk.StringVar()

    E1 = tk.Entry(knn2, textvariable = var, bd = 5, width = 100)
    E1.pack(side = tk.LEFT)

    def userinput():
        global var
        a = E1.get()
        predict_final(a)

    b = tk.Button(knn2, text = 'Submit', command = userinput, bg="#a1dbcd")
    b.pack(side = tk.BOTTOM)

In [4]:
import cv2
import os
import glob
import numpy as np
import pandas as pd
from statistics import median
import matplotlib.image as mpimg      
import matplotlib.pyplot as plt        
import matplotlib.patches as mpatches  
from skimage import measure            
import scipy.ndimage as ndi            
from pylab import rcParams
from skimage.feature import corner_harris, corner_subpix, corner_peaks, CENSURE
rcParams['figure.figsize'] = (6, 6)
def predict_final(a):
    
    def convert_to_gray(path):
        image = cv2.imread(path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return gray_image

    def convert_to_binary(path):
        image = cv2.imread(path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        ret,thresh_img = cv2.threshold(gray_image,127,255,cv2.THRESH_BINARY)
        bit_img = cv2.bitwise_not(thresh_img)
        kernel = np.ones((5,5),np.uint8)
        rem_noise = cv2.morphologyEx(bit_img, cv2.MORPH_OPEN, kernel)
        smooth_img = cv2.medianBlur(rem_noise, 5)
        return smooth_img

    def get_centre(img):
        cy, cx = ndi.center_of_mass(img)
        return (cx, cy)

    def get_margins(img):
        detector = CENSURE()
        detector.detect(img)
        coords = corner_peaks(corner_harris(img), min_distance=5)
        x = []
        for i in coords:
            for j in i:
                x.append(j)
        if len(x)<30:
            for i in range(len(x)-1, 30):
                x.append(0)
        return(x[:30])

    def get_texture_attr(img):
        return(img.mean(), img.std(), median(img.flatten()))

    def get_uniformity(img):
        non = img.astype(np.float32)/255
        uni = img.astype(np.float32)/255
        blur_non = cv2.GaussianBlur(non, (11, 11), 2)
        blur_uni = cv2.GaussianBlur(uni, (11, 11), 2)
        for i in range(10):
            blur_non = cv2.GaussianBlur(blur_non, (11, 11), 2)
            blur_uni = cv2.GaussianBlur(blur_uni, (11, 11), 2)

        last_blur_non = cv2.GaussianBlur(blur_non, (11, 11), 2)
        last_blur_uni = cv2.GaussianBlur(blur_uni, (11, 11), 2)

        ssd_blur_non = np.sum((last_blur_non - blur_non)**2)
        ssd_blur_uni = np.sum((last_blur_uni - blur_uni)**2)

        return(ssd_blur_non, ssd_blur_uni)

    def randomForestClassifier(path):
        feature_file = pd.read_csv('train_data.csv')
        label_file = pd.read_csv('test_data.csv')
        features = feature_file.values.tolist()
        label = label_file
        from sklearn.model_selection import train_test_split
        features_train, features_test, label_train, label_test = train_test_split(features, label, test_size=0.33)

        from sklearn.ensemble import RandomForestClassifier
        clf = RandomForestClassifier(min_samples_split=5,n_estimators=100)
        clf.fit(features_train, label_train)
        img1 = convert_to_gray(path)
        img2 = convert_to_binary(path)
        cv2.imwrite('output_gray.jpg', img1)
        cv2.imwrite('output_binary.jpg', img2)
        a, c, b = get_texture_attr(img1)
        d, e = get_uniformity(img1)
        f, g = get_centre(img2)
        h = get_margins(img2)
        test = [a,b,c,d,e,f,g]
        for i in range(30):
            test.append(h[i])
        output = clf.predict([test])

        return(output)

    species = ['Quercus suber', 'Salix atrocinerea', 'Populus nigra', 'Alnus sp', 'Quercus robur', 'Crataegus monogyna', 'Ilex aquifolium', 'Nerium oleander', 'Betula pubescens', 'Tilia tomentosa', 'Acer palmatum', 'Celtis sp.', 'Corylus avellana', 'Castanea sativa', 'Populus alba', 'Acer negundo', 'Taxus bacatta', 'Papaver sp.', 'Polypolium vulgare', 'Pinus sp.', 'Fraxinus sp.', 'Primula vulgaris', 'Erodium sp.', 'Bougainvillea sp.', 'Arisarum vulgare', 'Euonymus japonicus', 'Ilex perado ssp. azorica', 'Magnolia soulangeana', 'Buxus sempervirens', 'Urtica dioica', 'Podocarpus sp.', 'Acca sellowiana', 'Hydrangea sp.', 'Pseudosasa japonica', 'Magnolia grandiora','Geranium sp.','Aesculus californica','Chelidonium majus','Schinus terebinthifolius','Fragaria vesca']

    path = a
    [species_index] = randomForestClassifier(path)
    print('The plant in the given image belongs to',species[species_index-1], 'species')
    image_file = species[species_index-1] + ".jpg"
  
    from PIL import Image,ImageTk  
    rootimg = tk.Tk() 
    rootimg.geometry("100x100")
    rootimg.resizable(0,0)
    rootimg.title(species[species_index-1])  
    image=Image.open(image_file)  
    im = image.resize((350, 300), Image.ANTIALIAS)
    photo=ImageTk.PhotoImage(im)  
    
    cv = tk.Canvas(height=200, width=100)
    cv.pack(side='top', fill='both', expand='yes')  
    cv.create_image(150, 20, image=photo, anchor='nw')  
    rootimg.mainloop()

In [5]:
# Example of Naive Bayes implemented from Scratch in Python
import csv
import random
import math
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics 
def bayes():
    def loadCsv(filename):
        lines = csv.reader(open(filename, "r"))
        #next(reader, None)
        dataset = list(lines)
        for i in range(len(dataset)):
            dataset[i] = [float(x) for x in dataset[i]]
        return dataset

    def splitDataset(dataset, splitRatio):
        trainSize = int(len(dataset) * splitRatio)
        trainSet = []
        copy = list(dataset)
        while len(trainSet) < trainSize:
            index = random.randrange(len(copy))
            trainSet.append(copy.pop(index))
        return [trainSet, copy]

    def separateByClass(dataset):
        separated = {}
        for i in range(len(dataset)):
            vector = dataset[i]
            if (vector[-1] not in separated):
                separated[vector[-1]] = []
            separated[vector[-1]].append(vector)
        return separated

    def mean(numbers):
        return sum(numbers)/float(len(numbers))

    def stdev(numbers):
        avg = mean(numbers)
        variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
        return math.sqrt(variance)

    def summarize(dataset):
        summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
        del summaries[-1]
        return summaries

    def summarizeByClass(dataset):
        separated = separateByClass(dataset)
        summaries = {}
        for classValue, instances in separated.items():
            summaries[classValue] = summarize(instances)
        return summaries

    def calculateProbability(x, mean, stdev):
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

    def calculateClassProbabilities(summaries, inputVector):
        probabilities = {}
        for classValue, classSummaries in summaries.items():
            probabilities[classValue] = 1
            for i in range(len(classSummaries)):
                mean, stdev = classSummaries[i]
                x = inputVector[i]
                probabilities[classValue] *= calculateProbability(x, mean, stdev)
        return probabilities

    def predict(summaries, inputVector):
        probabilities = calculateClassProbabilities(summaries, inputVector)
        bestLabel, bestProb = None, -1
        for classValue, probability in probabilities.items():
            if bestLabel is None or probability > bestProb:
                bestProb = probability
                bestLabel = classValue
        return bestLabel

    def expect(testSet):
        expected=[]
        for i in range(len(testSet)):
            expected.append(testSet[i][-1])
        return expected

    def getPredictions(summaries, testSet):
        predictions = []
        for i in range(len(testSet)):
            result = predict(summaries, testSet[i])
            predictions.append(result)
        return predictions

    def getAccuracy(testSet, predictions):
        correct = 0
        for i in range(len(testSet)):
            if testSet[i][-1] == predictions[i]:
                correct += 1
        return (correct*2/float(len(testSet))) * 100.0 


    filename = 'leaf.csv'
    splitRatio = 0.67
    expected1=[]
    predictions1=[]
    cm=[]
    accuracy1=[]
    dataset = loadCsv(filename)
    check = ''
    for i in range(5):
        trainingSet, testSet = splitDataset(dataset, splitRatio)
        #print(('Split {0} rows into train={1} and test={2} rows').format(len(dataset), len(trainingSet), len(testSet)))

        # prepare model
        summaries = summarizeByClass(trainingSet)
        expected= expect(testSet)
        expected1.append(expected)
        # test model
        predictions = getPredictions(summaries, testSet)
        predictions1.append(predictions)
        accuracy =getAccuracy(testSet, predictions)
        accuracy1.append(accuracy)
        check += "Split Ratio = " + str("{0:.2f}".format(splitRatio)) + " : " + str("{0:.2f}".format(accuracy)) + " %\n"
        splitRatio += 0.05

    for i in range(5):
        if(accuracy1[i] == max(accuracy1)):
            maximum = i

    #print(('prediction: {0}').format(predictions1))
    #print(('expected: {0}').format(expected1))
    matrix = []
    for i in range(40):
        l = list()
        for j in range(40):
            l.append(0)
        matrix.append(l)
    for i in range(len(expected1[maximum])):
        matrix[int(expected1[maximum][i] - 1)][int(predictions1[maximum][i] - 1)] += 1
    check2 = ''
    for i in matrix:
        check2 += str(i) + "\n"

    count=1

    from tkinter import messagebox
 
    messagebox.showinfo("Confusion Matrix", check2)

    nb = tk.Tk()
    nb.geometry("300x400")
    nb.resizable(0,0)
    v = tk.IntVar()

    languages = [
        ("Test",1),
    ]

    def ShowChoice():
        if (v.get() == 0):
            image_accept()
         

    tk.Label(nb, 
             text=check,
             justify = tk.CENTER, fg="#383a39", bg="#a1dbcd", font=("Helvetica", 16),
             padx = 20).pack()

    for val, language in enumerate(languages):
        tk.Radiobutton(nb, 
                      text=language,
                      indicatoron = 0,
                      width = 100,
                      padx = 20, 
                      variable=v, 
                      command=ShowChoice,
                      value=val).pack(anchor=tk.W)
    nb.mainloop()


In [6]:
from sklearn import svm
import pandas as pd
from sklearn.metrics import accuracy_score
"""SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)"""
def supportVector():

    #read the csv files for both training and testing
    f = pd.read_csv('train2.csv')
    l = pd.read_csv('test2.csv')

    #creating data frames for training purposes
    #x will take all the features and y will take the labels
    x = f.values.tolist()
    y = list(l['leaf_class'])

    #creating test and training sets
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42)

    clf = svm.SVC()
    clf.fit(x_train, y_train)
    output = clf.predict(x_test)
    check = "Accuracy : " + str("{0:.2f}".format(accuracy_score(y_test, output) * 2000)) + "\n"
    matrix = []
    for i in range(40):
        l = list()
        for j in range(40):
            l.append(0)
        matrix.append(l)
    for i in range(len(y_test)):
        matrix[int(y_test[i] - 1)][int(output[i] - 1)] += 1

    check2 = ''
    for i in matrix:
        check2 += str(i) + "\n"

    from tkinter import messagebox
 
    messagebox.showinfo("Confusion Matrix", check2)

    sv = tk.Tk()
    sv.geometry("300x400")
    sv.resizable(0,0)
    v = tk.IntVar()

    languages = [
        ("Test",1),
    ]

    def ShowChoice():
        if (v.get() == 0):
            image_accept()
         

    tk.Label(sv, 
             text=check,
             justify = tk.CENTER, fg="#383a39", bg="#a1dbcd", font=("Helvetica", 16),
             padx = 20).pack()

    for val, language in enumerate(languages):
        tk.Radiobutton(sv, 
                      text=language,
                      indicatoron = 0,
                      width = 100,
                      padx = 20, 
                      variable=v, 
                      command=ShowChoice,
                      value=val).pack(anchor=tk.W)
    sv.mainloop()

In [10]:
import tkinter as tk
from tkinter import messagebox

root = tk.Tk()
root.geometry("600x600")
root.resizable(0, 0)
v = tk.IntVar()

languages = [
    ("ANN",1),
    ("kNN",2),
    ("Bayes",3),
    ("SVM",4),
]

def ShowChoice():
    if(v.get() == 0):
        ANN()
    elif(v.get() == 1):
        kNN()
    elif(v.get() == 2):
        bayes()
    else:
        supportVector()

tk.Label(root, 
         text="""Choose the algorithm you wish to train the model with:""",fg="#383a39", bg="#a1dbcd", font=("Helvetica", 16),
         justify = tk.CENTER,
         padx = 20).pack()

for val, language in enumerate(languages):
    tk.Radiobutton(root, 
                  text=language,
                  indicatoron = 0,
                  width = 85,
                  padx = 20, 
                  variable=v, 
                  command=ShowChoice,
                  value=val).pack(anchor=tk.W)


root.mainloop()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\user\Anaconda3\lib\statistics.py:385: RuntimeWarning: overflow encountered in ubyte_scalars
  return (data[i - 1] + data[i])/2


The plant in the given image belongs to Salix atrocinerea species
